Data "Hourly energy demand generation and weather"

Preprocessing Data

In [3]:
import pandas as pd

# Baca Data
df_energy = pd.read_csv('D:/kuliah/kodingan/github/EnergyDemand/energy_dataset.csv', parse_dates=['time'])
df_weather = pd.read_csv('D:/kuliah/kodingan/github/EnergyDemand/weather_features.csv', parse_dates=['dt_iso'])

# Ubah nama kolom waktu agar sama
df_weather.rename(columns={'dt_iso': 'time'}, inplace=True)

# Pilih hanya kolom numerik untuk agregasi
numeric_weather_cols = df_weather.select_dtypes(include='number').columns

# Gabungkan berdasarkan waktu (rata-rata per jam untuk tiap kota)
weather_hourly = df_weather.groupby('time')[numeric_weather_cols].mean().reset_index()

# Gabungkan data energy dan cuaca
df = pd.merge(df_energy, weather_hourly, on='time', how='inner')

# Isi missing values
df.fillna(method='ffill', inplace=True)


C:\Users\ACER\AppData\Local\Temp\ipykernel_5868\755640121.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [10]:
df.columns

Index(['time', 'generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat',
       'generation geothermal', 'generation hydro pumped storage aggregated',
       'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
       'generation nuclear', 'generation other', 'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore', 'forecast solar day ahead',
       'forecast wind offshore eday ahead', 'forecast wind onshore day ahead',
       'total load forecast', 'total load actual', 'price day ahead',
       'price actual', 'temp', 'temp_min', 'temp_max', 'pressure', 'humidity',
       'wind_speed', 'wind_deg', 

In [12]:
# Pastikan kolom 'time' bertipe datetime
df['time'] = pd.to_datetime(df['time'], utc=True).dt.tz_localize(None)


# Feature creation: extract time-based features
df['hour'] = df['time'].dt.hour
df['dayofweek'] = df['time'].dt.dayofweek
df['month'] = df['time'].dt.month

# Renewable energy generation total (sum across selected sources)
renewable_sources = ['generation solar', 'generation wind onshore', 'generation wind offshore', 'generation hydro run-of-river and poundage']
df['renewable_total'] = df[renewable_sources].sum(axis=1)

# Total generation from all energy sources (manual selection)
generation_columns = [col for col in df.columns if col.startswith('generation_') and not 'consumption' in col]
df['total_generation'] = df[generation_columns].sum(axis=1)

# Renewable share
df['renewable_share'] = df['renewable_total'] / df['total_generation']


In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Load forecast vs actual
y_true = df['total load actual']
y_pred = df['total load forecast']
mae_load = mean_absolute_error(y_true, y_pred)
rmse_load = np.sqrt(mean_squared_error(y_true, y_pred))

# Solar forecast accuracy
y_true_solar = df['generation solar']
y_pred_solar = df['forecast solar day ahead']
mae_solar = mean_absolute_error(y_true_solar, y_pred_solar)

KeyError: 'generation_solar'

In [5]:
#menampilkan data kolom
df_energy.columns

Index(['time', 'generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat',
       'generation geothermal', 'generation hydro pumped storage aggregated',
       'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
       'generation nuclear', 'generation other', 'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore', 'forecast solar day ahead',
       'forecast wind offshore eday ahead', 'forecast wind onshore day ahead',
       'total load forecast', 'total load actual', 'price day ahead',
       'price actual'],
      dtype='object')

In [6]:
df_weather.columns

Index(['dt_iso', 'city_name', 'temp', 'temp_min', 'temp_max', 'pressure',
       'humidity', 'wind_speed', 'wind_deg', 'rain_1h', 'rain_3h', 'snow_3h',
       'clouds_all', 'weather_id', 'weather_main', 'weather_description',
       'weather_icon'],
      dtype='object')

In [7]:
#Info type data per columns
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178396 entries, 0 to 178395
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   dt_iso               178396 non-null  object 
 1   city_name            178396 non-null  object 
 2   temp                 178396 non-null  float64
 3   temp_min             178396 non-null  float64
 4   temp_max             178396 non-null  float64
 5   pressure             178396 non-null  int64  
 6   humidity             178396 non-null  int64  
 7   wind_speed           178396 non-null  int64  
 8   wind_deg             178396 non-null  int64  
 9   rain_1h              178396 non-null  float64
 10  rain_3h              178396 non-null  float64
 11  snow_3h              178396 non-null  float64
 12  clouds_all           178396 non-null  int64  
 13  weather_id           178396 non-null  int64  
 14  weather_main         178396 non-null  object 
 15  weather_descripti

In [8]:
df_energy.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 29 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   time                                         35064 non-null  object 
 1   generation biomass                           35045 non-null  float64
 2   generation fossil brown coal/lignite         35046 non-null  float64
 3   generation fossil coal-derived gas           35046 non-null  float64
 4   generation fossil gas                        35046 non-null  float64
 5   generation fossil hard coal                  35046 non-null  float64
 6   generation fossil oil                        35045 non-null  float64
 7   generation fossil oil shale                  35046 non-null  float64
 8   generation fossil peat                       35046 non-null  float64
 9   generation geothermal                        35046 non-null  float64
 10

Data